In [ ]:
import marimo as mo
import cyvcf2
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
def read_vcf_data():
    # Read the VCF file with pre-calculated DP and VAF
    vcf_path = "xhetrel/chrX.vcf.gz"
    reader = cyvcf2.Reader(vcf_path)

    # Dictionary to store genotype counts per sample for different depth ranges and VAF filters
    sample_genotypes = {
        'dp_0_10_vaf': {sample: Counter() for sample in reader.samples},
        'dp_0_10_no_vaf': {sample: Counter() for sample in reader.samples},
        'dp_10_20_vaf': {sample: Counter() for sample in reader.samples},
        'dp_10_20_no_vaf': {sample: Counter() for sample in reader.samples},
        'dp_20_plus_vaf': {sample: Counter() for sample in reader.samples},
        'dp_20_plus_no_vaf': {sample: Counter() for sample in reader.samples}
    }

    # Process variants
    for variant in reader:
        # Process each sample
        for i, sample in enumerate(reader.samples):
            # Get DP and VAF
            dp = variant.format('DP')[i][0]
            vaf_values = variant.format('VAF')

            # Skip if VAF is not available or NaN
            if vaf_values is None or np.isnan(vaf_values[i][0]):
                continue

            vaf = vaf_values[i][0]
            gt_type = variant.gt_types[i]

            # Skip UNKNOWN genotypes
            if gt_type == 2:
                continue

            # Process DP 0-10 range
            if 0 <= dp < 10:
                # With VAF filter
                if vaf >= 0.25:
                    if gt_type == 1:  # HET
                        sample_genotypes['dp_0_10_vaf'][sample]["0/1"] += 1
                    elif gt_type == 3:  # HOM_ALT
                        sample_genotypes['dp_0_10_vaf'][sample]["1/1"] += 1
                # Without VAF filter
                if gt_type == 1:  # HET
                    sample_genotypes['dp_0_10_no_vaf'][sample]["0/1"] += 1
                elif gt_type == 3:  # HOM_ALT
                    sample_genotypes['dp_0_10_no_vaf'][sample]["1/1"] += 1

            # Process DP 10-20 range
            elif 10 <= dp < 20:
                # With VAF filter
                if vaf >= 0.25:
                    if gt_type == 1:  # HET
                        sample_genotypes['dp_10_20_vaf'][sample]["0/1"] += 1
                    elif gt_type == 3:  # HOM_ALT
                        sample_genotypes['dp_10_20_vaf'][sample]["1/1"] += 1
                # Without VAF filter
                if gt_type == 1:  # HET
                    sample_genotypes['dp_10_20_no_vaf'][sample]["0/1"] += 1
                elif gt_type == 3:  # HOM_ALT
                    sample_genotypes['dp_10_20_no_vaf'][sample]["1/1"] += 1

            # Process DP ≥20 range
            elif dp >= 20:
                # With VAF filter
                if vaf >= 0.25:
                    if gt_type == 1:  # HET
                        sample_genotypes['dp_20_plus_vaf'][sample]["0/1"] += 1
                    elif gt_type == 3:  # HOM_ALT
                        sample_genotypes['dp_20_plus_vaf'][sample]["1/1"] += 1
                # Without VAF filter
                if gt_type == 1:  # HET
                    sample_genotypes['dp_20_plus_no_vaf'][sample]["0/1"] += 1
                elif gt_type == 3:  # HOM_ALT
                    sample_genotypes['dp_20_plus_no_vaf'][sample]["1/1"] += 1

    return sample_genotypes

def read_sample_sex():
    # Read sample sex information
    sex_path = "xhetrel/sample_sex"
    sex_df = pd.read_csv(sex_path, sep='\t', names=['Sample', 'Sex'])

    # Translate Turkish sex labels to English
    sex_translation = {
        'Kadın': 'female',
        'Erkek': 'male'
    }
    sex_df['Sex'] = sex_df['Sex'].map(sex_translation)

    # Remove samples with missing sex information
    sex_df = sex_df.dropna(subset=['Sex'])

    return sex_df


In [ ]:
# Get the data and calculate ratios for each depth range and VAF filter
genotype_counts = read_vcf_data()
sex_df = read_sample_sex()

def calculate_het_ratios(genotype_counts, sex_df):
    # Calculate heterozygosity ratio for each sample
    results = []
    for sample, counts in genotype_counts.items():
        het_count = counts.get("0/1", 0)
        hom_alt_count = counts.get("1/1", 0)
        total_het_hom = het_count + hom_alt_count

        if total_het_hom > 0:
            het_ratio = het_count / total_het_hom
            results.append({
                'Sample': sample,
                'Heterozygous_Ratio': het_ratio,
                'Total_Variants': sum(counts.values()),
                'Het_Variants': het_count,
                'Hom_Alt_Variants': hom_alt_count
            })

    # Create DataFrame and merge with sex information
    results_df = pd.DataFrame(results)
    results_df = results_df.merge(sex_df, on='Sample', how='inner')
    results_df = results_df.sort_values('Heterozygous_Ratio', ascending=False)
    return results_df

het_ratios_dfs = {
    'DP 0-10 (VAF ≥ 0.25)': calculate_het_ratios(genotype_counts['dp_0_10_vaf'], sex_df),
    'DP 0-10 (No VAF filter)': calculate_het_ratios(genotype_counts['dp_0_10_no_vaf'], sex_df),
    'DP 10-20 (VAF ≥ 0.25)': calculate_het_ratios(genotype_counts['dp_10_20_vaf'], sex_df),
    'DP 10-20 (No VAF filter)': calculate_het_ratios(genotype_counts['dp_10_20_no_vaf'], sex_df),
    'DP ≥20 (VAF ≥ 0.25)': calculate_het_ratios(genotype_counts['dp_20_plus_vaf'], sex_df),
    'DP ≥20 (No VAF filter)': calculate_het_ratios(genotype_counts['dp_20_plus_no_vaf'], sex_df)
}

# Print summary for each condition
for condition, df in het_ratios_dfs.items():
    print(f"\n{condition}:")
    print(f"Total samples: {len(df)}")
    print(f"Mean heterozygous ratio: {df['Heterozygous_Ratio'].mean():.3f}")
    print(f"Total variants: {df['Total_Variants'].sum()}")


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def plot_analysis_results(df, depth_range, model, X_test, y_test, y_pred_proba):
    """Plot various analysis results including scatter plot, ROC curve, precision-recall curve, and threshold analysis.

    Args:
        df: DataFrame containing the data
        depth_range: String describing the depth range and variant percentage
        model: Trained logistic regression model
        X_test: Test features
        y_test: True labels for test set
        y_pred_proba: Predicted probabilities for test set
    """
    # Plot scatter plot with decision boundary
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Heterozygous_Ratio', y='Sex', alpha=0.5, hue='Sex')

    # Plot decision boundary
    x_range = np.linspace(df['Heterozygous_Ratio'].min(), 
                        df['Heterozygous_Ratio'].max(), 100)
    y_range = model.predict_proba(x_range.reshape(-1, 1))[:, 1]
    plt.plot(x_range, y_range, 'r-', label='Decision Boundary')

    plt.title(f'Sex Prediction using Heterozygous Ratio ({depth_range})')
    plt.xlabel('Heterozygous Ratio')
    plt.ylabel('Sex')
    plt.legend()
    plt.show()

    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)

    # Plot ROC curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve ({depth_range})')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Find optimal threshold (Youden's J statistic)
    J = tpr - fpr
    optimal_idx = np.argmax(J)
    optimal_threshold = thresholds[optimal_idx]

    # Calculate precision and recall for different thresholds
    precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred_proba)

    # Plot precision-recall curve
    plt.figure(figsize=(10, 6))
    plt.plot(recall, precision, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve ({depth_range})')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot threshold analysis
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, tpr, label='True Positive Rate')
    plt.plot(thresholds, 1-fpr, label='True Negative Rate')
    plt.axvline(x=optimal_threshold, color='r', linestyle='--', 
                label=f'Optimal threshold: {optimal_threshold:.3f}')
    plt.xlabel('Threshold')
    plt.ylabel('Rate')
    plt.title(f'Threshold Analysis ({depth_range})')
    plt.legend()
    plt.grid(True)
    plt.show()

    return optimal_threshold

def analyze_depth_range(df, depth_range, variant_percentage=None, plot_results=True):
    """Analyze heterozygous ratios for a given depth range and variant percentage.

    Args:
        df: DataFrame containing the data
        depth_range: String describing the depth range
        variant_percentage: Optional float between 0 and 1 to select a random subset of variants
        plot_results: Boolean to control whether to generate plots

    Returns:
        tuple: (accuracy, auc, mw_pval, t_pval, cohens_d, optimal_threshold, avg_female_variants, avg_male_variants) metrics
    """
    # If variant_percentage is specified, randomly select that percentage of variants
    if variant_percentage is not None:
        # Calculate how many variants to keep for each sample
        df = df.copy()

        # Calculate valid variants (sum of het and hom_alt)
        df['Valid_Variants'] = df['Het_Variants'] + df['Hom_Alt_Variants']

        # Use a different random seed for each sample based on the sample name
        df['Selected_Variants'] = df.apply(
            lambda row: max(1, np.random.RandomState(hash(row['Sample']) % 2**32).binomial(
                row['Valid_Variants'], 
                variant_percentage
            )),
            axis=1
        )

        # Print information about variant selection
        #print("\nVariant selection summary:")
        #for _, row in df.iterrows():
        #    print(f"{row['Sample']}: Selected {row['Selected_Variants']} from {row['Valid_Variants']} valid variants ({row['Selected_Variants']/row['Valid_Variants']*100:.1f}%)")

        # Recalculate heterozygous ratio with selected variants
        df['Heterozygous_Ratio'] = df['Het_Variants'] * variant_percentage / df['Selected_Variants']

        depth_range = f"{depth_range} ({variant_percentage*100:.0f}% variants)"

    # Calculate average variants by sex
    avg_female_variants = df[df['Sex'] == 'female']['Selected_Variants'].mean()
    avg_male_variants = df[df['Sex'] == 'male']['Selected_Variants'].mean()

    # Print data distribution
    print(f"\n{depth_range} Data Distribution:")
    print(f"Total samples: {len(df)}")
    print(f"Number of females: {len(df[df['Sex'] == 'female'])}")
    print(f"Number of males: {len(df[df['Sex'] == 'male'])}")
    print(f"Average variants - Females: {avg_female_variants:.1f}, Males: {avg_male_variants:.1f}")

    # Calculate and print statistics by sex
    female_ratios = df[df['Sex'] == 'female']['Heterozygous_Ratio']
    male_ratios = df[df['Sex'] == 'male']['Heterozygous_Ratio']

    print("\nHeterozygous Ratio Statistics by Sex:")
    print("\nFemales:")
    print(female_ratios.describe())
    print("\nMales:")
    print(male_ratios.describe())

    # Perform statistical tests
    from scipy import stats

    # Mann-Whitney U test (non-parametric test for comparing two independent samples)
    mw_stat, mw_pval = stats.mannwhitneyu(female_ratios, male_ratios, alternative='two-sided')

    # T-test (parametric test)
    t_stat, t_pval = stats.ttest_ind(female_ratios, male_ratios)

    # Effect size (Cohen's d)
    cohens_d = (female_ratios.mean() - male_ratios.mean()) / np.sqrt((female_ratios.var() + male_ratios.var()) / 2)

    print("\nStatistical Tests:")
    print(f"Mann-Whitney U test p-value: {mw_pval:.2e}")
    print(f"T-test p-value: {t_pval:.2e}")
    print(f"Effect size (Cohen's d): {cohens_d:.3f}")

    print("\nTotal Variants Statistics:")
    print(df['Total_Variants'].describe())

    # Prepare data for logistic regression
    X = df[['Heterozygous_Ratio']]
    y = (df['Sex'] == 'female').astype(int)  # Convert to binary (1 for female, 0 for male)

    # Perform cross-validation
    model = LogisticRegression()
    cv_scores = cross_val_score(model, X, y, cv=5)
    print(f"\nCross-validation scores: {cv_scores}")
    print(f"Mean CV score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit logistic regression
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    # Calculate optimal threshold using ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    J = tpr - fpr
    optimal_idx = np.argmax(J)
    optimal_threshold = thresholds[optimal_idx]

    # Print results
    print(f"\n{depth_range} Analysis:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"AUC-ROC: {auc:.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Generate plots if requested
    if plot_results:
        plot_analysis_results(df, depth_range, model, X_test, y_test, y_pred_proba)
        print("\nDecision Thresholds Analysis:")
        print(f"Default threshold (0.5): {model.predict_proba(X_test)[:, 1].mean():.3f}")
        print(f"Optimal threshold (Youden's J): {optimal_threshold:.3f}")

    return accuracy, auc, mw_pval, t_pval, cohens_d, optimal_threshold, avg_female_variants, avg_male_variants

# Analyze each condition with different variant percentages
results = {}
for depth_range_, df_ in het_ratios_dfs.items():
    print(f"\n{'='*80}")
    print(f"Analyzing {depth_range_}")
    print(f"{'='*80}")

    # Analyze with different percentages of variants
    percentages = [1.0, 0.75, 0.5, 0.25, 0.10, 0.05, 0.01]
    results[depth_range_] = {}

    for pct in percentages:
        print(f"{'*'*80}")
        print(f"\nAnalyzing with {pct*100:.0f}% of variants")
        accuracy, auc, mw_pval, t_pval, cohens_d, optimal_threshold, avg_female_variants, avg_male_variants = analyze_depth_range(df_, depth_range_, pct, plot_results=True)
        print(f"{'*'*80}")
        results[depth_range_][pct] = {
            'accuracy': accuracy, 
            'auc': auc, 
            'mw_pval': mw_pval, 
            't_pval': t_pval, 
            'cohens_d': cohens_d,
            'optimal_threshold': optimal_threshold,
            'avg_female_variants': avg_female_variants,
            'avg_male_variants': avg_male_variants
        }


In [ ]:
def create_metrics_table(results):
    """Create a pandas DataFrame with all metrics data."""
    # Create a list to store all the data
    table_data = []

    # Iterate through the results
    for depth_range, metrics in results.items():
        for pct, values in metrics.items():
            row = {
                'Depth Range': depth_range,
                'Variant Percentage': f"{pct*100:.0f}%",
                'Accuracy': f"{values['accuracy']:.3f}",
                'AUC-ROC': f"{values['auc']:.3f}",
                'Optimal Threshold': f"{values['optimal_threshold']:.3f}",
                'Avg Female Variants': f"{values['avg_female_variants']:.1f}",
                'Avg Male Variants': f"{values['avg_male_variants']:.1f}",
                'Mann-Whitney p-value': f"{values['mw_pval']:.2e}",
                'T-test p-value': f"{values['t_pval']:.2e}",
                "Cohen's d": f"{values['cohens_d']:.3f}"
            }
            table_data.append(row)

    # Create DataFrame and sort by depth range and variant percentage
    df = pd.DataFrame(table_data)
    df = df.sort_values(['Depth Range', 'Variant Percentage'])
    print(df)

create_metrics_table(results)

In [ ]:
def plot_comparison_metrics(results):
    """Plot comparison metrics across different depth ranges and variant percentages."""
    # Set font to DejaVu Sans
    plt.rcParams['font.family'] = 'DejaVu Sans'

    # Define color palette
    colors = {
        'red': '#FA5252',  # red5
        'blue': '#4DABF7',  # blue5
        'violet': '#845EF7',  # violet5
        'indigo': '#5C7CFA',  # indigo5
        'pink': '#F06595',  # pink5
        'teal': '#20C997',  # teal5
        'orange': '#FF922B',  # orange5
        'cyan': '#22B8CF',  # cyan5
        'grape': '#CC5DE8',  # grape5
        'green': '#51CF66',  # green5
        'yellow': '#FCC419',  # yellow5
        'lime': '#94D82D',  # lime5
    }

    # Define line styles for different depth ranges
    line_styles = ['-', '--', '-.', ':']
    markers = ['o', 's', '^', 'D']

    plt.figure(figsize=(15, 12))

    # Plot accuracy
    ax1 = plt.subplot(221)
    for i, (depth_range, metrics) in enumerate(results.items()):
        percentages = list(metrics.keys())
        accuracies = [m['accuracy'] for m in metrics.values()]
        color = list(colors.values())[i % len(colors)]
        style = line_styles[i % len(line_styles)]
        marker = markers[i % len(markers)]
        plt.plot(percentages, accuracies, 
                marker=marker, 
                linestyle=style, 
                color=color, 
                alpha=0.8,
                linewidth=2,
                label=depth_range)
    plt.xlabel('Percentage of Variants')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Variant Percentage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.text(-0.1, 1.1, 'A', transform=ax1.transAxes, fontsize=16, fontweight='bold')

    # Plot AUC-ROC
    ax2 = plt.subplot(222)
    for i, (depth_range, metrics) in enumerate(results.items()):
        percentages = list(metrics.keys())
        aucs = [m['auc'] for m in metrics.values()]
        color = list(colors.values())[i % len(colors)]
        style = line_styles[i % len(line_styles)]
        marker = markers[i % len(markers)]
        plt.plot(percentages, aucs, 
                marker=marker, 
                linestyle=style, 
                color=color, 
                alpha=0.8,
                linewidth=2,
                label=depth_range)
    plt.xlabel('Percentage of Variants')
    plt.ylabel('AUC-ROC')
    plt.title('AUC-ROC vs. Variant Percentage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.text(-0.1, 1.1, 'B', transform=ax2.transAxes, fontsize=16, fontweight='bold')

    # Plot optimal thresholds
    ax3 = plt.subplot(223)
    for i, (depth_range, metrics) in enumerate(results.items()):
        percentages = list(metrics.keys())
        thresholds = [m['optimal_threshold'] for m in metrics.values()]
        color = list(colors.values())[i % len(colors)]
        style = line_styles[i % len(line_styles)]
        marker = markers[i % len(markers)]
        plt.plot(percentages, thresholds, 
                marker=marker, 
                linestyle=style, 
                color=color, 
                alpha=0.8,
                linewidth=2,
                label=depth_range)
    plt.xlabel('Percentage of Variants')
    plt.ylabel('Optimal Threshold')
    plt.title('Optimal Threshold vs. Variant Percentage')
    plt.legend()
    plt.grid(True, alpha=0.3)
    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.text(-0.1, 1.1, 'C', transform=ax3.transAxes, fontsize=16, fontweight='bold')

    # Plot average variants by sex
    ax4 = plt.subplot(224)

    # Plot all variants with distinct styles for each depth range
    for i, (depth_range, metrics) in enumerate(results.items()):
        percentages = list(metrics.keys())
        female_variants = [m['avg_female_variants'] for m in metrics.values()]
        male_variants = [m['avg_male_variants'] for m in metrics.values()]
        color = list(colors.values())[i % len(colors)]
        style = line_styles[i % len(line_styles)]
        marker = markers[i % len(markers)]

        # Plot female variants
        plt.plot(percentages, female_variants, 
                marker=marker, 
                linestyle=style, 
                color=color, 
                alpha=0.8,
                linewidth=2,
                label=f'{depth_range} (Female)')

        # Plot male variants with same color but different style
        plt.plot(percentages, male_variants, 
                marker=marker, 
                linestyle=style, 
                color=color, 
                alpha=0.4,
                linewidth=1.5,
                label=f'{depth_range} (Male)')

    plt.xlabel('Percentage of Variants')
    plt.ylabel('Average Number of Variants')
    plt.title('Average Variants by Sex')
    plt.legend()
    plt.grid(True, alpha=0.3)
    ax4.spines['top'].set_visible(False)
    ax4.spines['right'].set_visible(False)
    ax4.text(-0.1, 1.1, 'D', transform=ax4.transAxes, fontsize=16, fontweight='bold')

    plt.tight_layout()
    plt.show()

plot_comparison_metrics(results)